In [1]:
!pip install --upgrade pip

In [4]:
!pip install tensorflow
!pip install opencv-python    
!pip install pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 2.1 MB/s eta 0:00:00m eta 0:00:010:00:01


In [1]:
import cv2
import glob
import random as ra
import PIL
from PIL import Image
from numpy import asarray

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras import layers
from keras import utils
import sklearn.metrics as metrics
import numpy as np
from tensorflow.keras import regularizers
import math

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping, ModelCheckpoint

def process(X,file):
    img=tf.keras.preprocessing.image.load_img(
        file,color_mode="grayscale", target_size=None, interpolation="nearest"
    )
    img=cv2.imread(file)
    X.append(img)

Normal=[]
Bacterial=[]
Virus=[]
classes=[Virus,Bacterial,Normal]
Y_Normal=[]
Y_Bacterial=[]
Y_Virus=[]
Y=[Y_Normal,Y_Bacterial,Y_Virus]
files=glob.glob('chest_xray_lower_dim2'+'/*.jpeg')
ra.shuffle(files)
for file in files:
    k=-1
    if "virus" in file:
        k=0
    elif "bacteria" in file:
        k=1
    else:
        k=2
    Y[k].append(str(k))
    process(classes[k],file)
    
len_classes=[]
for class_ in classes:
    len_classes.append(len(class_))
print(len_classes)
same_length=min(len_classes)
evenly=False

if evenly:
    for i in range(3):
        len_classes[i]=same_length

for i in range(3):
    classes[i]=classes[i][0:len_classes[i]]
for i in range(3):
    Y[i]=Y[i][0:len_classes[i]]

Train=[]
Valid=[]
Test=[]
TVT=[Train,Valid,Test]
Y_Train=[]
Y_Valid=[]
Y_Test=[]
Y_TVT=[Y_Train,Y_Valid,Y_Test]

bounds=[0,0.64,0.8,1]
for i in range(3):
    for j in range(3):
        TVT[i]+=classes[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
        Y_TVT[i]+=Y[j][int(bounds[i]*len_classes[j]):int(bounds[i+1]*len_classes[j])]
    Y_TVT[i]=keras.utils.to_categorical(Y_TVT[i],3)
    

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rescale=1./255,
        zoom_range=[-0.2,0.2],
        rotation_range=[-20,20],
        horizontal_flip=True)
validation_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen =tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_datagen.fit(Train)
validation_datagen.fit(Valid)
test_datagen.fit(Test)

def train(epochs_num,batch_len,conv_neurons,art_neurons):
    
    def mlp_model():
        Inp=layers.Input(shape=(100,100,3))
        hidden=Inp
        for conv_neuron in conv_neurons:
            hidden=layers.Conv2D(conv_neuron,kernel_size=(3,3),padding='valid',activation='relu')(hidden)
            hidden=layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(hidden)
        hidden=layers.Flatten()(hidden)
        inputs=100*100
        w=math.sqrt(inputs)
        for art_neuron in art_neurons:
            i2=i+1
            hidden=layers.Dense(art_neuron,activation='relu',kernel_regularizer=regularizers.L2(0.0001))(hidden)
            #hidden=layers.Dropout(0.2)(hidden)
            inputs=30
            w=math.sqrt(inputs)
        output=layers.Dense(3,activation='softmax',kernel_regularizer=regularizers.L2(0.0001))(hidden)
        model=keras.Model(Inp,output)
        model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
        return model
    """
    checkpoint = tf.keras.callbacks.ModelCheckpoint("weights.hdf5",monitor='val_accuracy', verbose=1, 
    save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    estimator = KerasClassifier(build_fn=mlp_model, epochs=int(len(X_train)/dividor_epoch), batch_size=batch_len, verbose=2)
    kfold = KFold(n_splits=8, shuffle=True, random_state=7)
    results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
    print("MLP: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    """
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=4,mode='max')
    model=mlp_model()
    model.fit(train_datagen.flow(TVT[0],Y_TVT[0],batch_size=32,subset='training'),
              validation_data=(TVT[1],Y_TVT[1]),epochs=epochs_num,
              batch_size=batch_len,callbacks=[callback],verbose=1)
    #model.load_weights("weights.hdf5")
    model.summary()
    score= model.evaluate(TVT[2], Y_TVT[2],verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:",score[1])
    
train(100,150,[32,64,96],[50,50])



   
        


2022-06-25 20:34:28.605235: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/manos/notebook/jupyterenv/lib/python3.10/site-packages/cv2/../../lib64:
2022-06-25 20:34:28.605298: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[1493, 2780, 1583]


2022-06-25 20:36:20.040460: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/manos/notebook/jupyterenv/lib/python3.10/site-packages/cv2/../../lib64:
2022-06-25 20:36:20.062502: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-25 20:36:20.076751: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (manos-Lenovo-G50-70): /proc/driver/nvidia/version does not exist
2022-06-25 20:36:20.552657: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: `x` (images tensor) and `y` (labels) should have the same length. Found: x.shape = (100, 100, 3), y.shape = (3747, 3)